# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions.

In [1]:
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

import numpy as np
import timeit as t
import base64
import json
from PIL import Image
from PIL import ImageOps
from io import BytesIO
import logging

Using TensorFlow backend.


Here, we develop the functions that will go into the module. Let's first load the model and define the number of top predictions to return.

In [2]:
%%time
model = ResNet152(weights='imagenet')

CPU times: user 16.2 s, sys: 905 ms, total: 17.1 s
Wall time: 16.8 s


In [3]:
number_results = 3

The API will expect the input to be in JSON format. Below we define the function to convert the image to json using the image path as key and the value as image encoded as a base64 string .

In [4]:
def img_to_json(img_path):
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    return body

In [5]:
img_path = '220px-Lynx_lynx_poing.jpg'
body = img_to_json(img_path)

Once  a request is received by the API, we will convert the json encoded request body into the image format.

In [6]:
base64Dict = json.loads(body) 
for k, v in base64Dict.items():
    img_file_name, base64Img = k, v 
decoded_img = base64.b64decode(base64Img)
img_buffer = BytesIO(decoded_img)
imageData = Image.open(img_buffer).convert("RGB")

We will then preprocess the image and score it using the model.

In [7]:
img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
preds = model.predict(img)
preds = decode_predictions(preds, top=number_results)[0]
resp = {img_path: preds}
resp

{'220px-Lynx_lynx_poing.jpg': [('n02127052', 'lynx', 0.98164833),
  ('n02128385', 'leopard', 0.0077441484),
  ('n02123159', 'tiger_cat', 0.0036861449)]}

In [12]:
def create_scoring_func():
    """ Initialize ResNet 152 Model 
    """   
    print("Executing create_scoring_func() method...")
    number_results = 3    
    logger = logging.getLogger("model_driver")

    start = t.default_timer()
    model = ResNet152(weights='imagenet')
    end = t.default_timer()
    
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    print("Model loading time: {0} ms".format(round((end-start)*1000, 2)))
    
    def call_model(img_array):
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        preds = decode_predictions(preds, top=number_results)[0]       
        return preds
    
    return call_model

In [9]:
def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        base64Dict = json.loads(inputString) 
        for k, v in base64Dict.items():
            img_file_name, base64Img = k, v 
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        imageData = Image.open(img_buffer).convert("RGB")
        img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
        img_array = image.img_to_array(img)
        preds = scoring_func(img_array)
        resp = {img_file_name: preds}
        responses.append(resp)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

Let's test our methods using the same Lynx image.

In [13]:
logging.basicConfig(level=logging.DEBUG)

In [14]:
predict_for = get_model_api()
img_path = '220px-Lynx_lynx_poing.jpg'
body = img_to_json(img_path)
resp = predict_for(body)
print(resp)

Executing create_scoring_func() method...


INFO:model_driver:Model loading time: 17247.84 ms


Model loading time: 17247.84 ms


INFO:model_driver:Predictions: [{'220px-Lynx_lynx_poing.jpg': [('n02127052', 'lynx', 0.98164833), ('n02128385', 'leopard', 0.0077441484), ('n02123159', 'tiger_cat', 0.0036861449)]}]
INFO:model_driver:Predictions took 2170.73 ms


([{'220px-Lynx_lynx_poing.jpg': [('n02127052', 'lynx', 0.98164833), ('n02128385', 'leopard', 0.0077441484), ('n02123159', 'tiger_cat', 0.0036861449)]}], 'Computed in 2170.73 ms')


We use the writefile magic to write the contents of the below cell which includes the driver methods we developed earlier to the file driver.py.

In [4]:
%%writefile driver.py

import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

import numpy as np
import timeit as t
import base64
import json
from PIL import Image, ImageOps
from io import BytesIO
import logging

number_results = 3
logger = logging.getLogger("model_driver")

def img_to_json(img_path):
    with open(img_path, 'rb') as file:
        encoded = base64.b64encode(file.read())
    img_dict = {img_path: encoded.decode('utf-8')}
    body = json.dumps(img_dict)
    return body

def create_scoring_func():
    """ Initialize ResNet 152 Model 
    """   
    print("Executing create_scoring_func() method...")
    
    start = t.default_timer()
    model = ResNet152(weights='imagenet')
    end = t.default_timer()
    
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    print("Model loading time: {0} ms".format(round((end-start)*1000, 2)))
    
    def call_model(img_array):
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        preds = decode_predictions(preds, top=number_results)[0]       
        return preds
    
    return call_model       

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        print("Executing process_and_score() method...")
        
        start = t.default_timer()

        responses = []
        base64Dict = json.loads(inputString) 
        for k, v in base64Dict.items():
            img_file_name, base64Img = k, v 
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        imageData = Image.open(img_buffer).convert("RGB")
        img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
        img_array = image.img_to_array(img)
        preds = scoring_func(img_array)
        resp = {img_file_name: preds}
        responses.append(resp)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Predictions took {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return tf.__version__

if __name__ == "__main__":  
    predict_for = get_model_api()
    img_path = '220px-Lynx_lynx_poing.jpg'
    body = img_to_json(img_path)
    resp = predict_for(body)
    print(resp)

Overwriting driver.py


Let's test the module.

In [19]:
%run driver.py

Executing create_scoring_func() method...


INFO:model_driver:Model loading time: 18125.59 ms


Model loading time: 18125.59 ms
Executing process_and_score() method...


INFO:model_driver:Predictions: [{'220px-Lynx_lynx_poing.jpg': [('n02127052', 'lynx', 0.98164833), ('n02128385', 'leopard', 0.0077441484), ('n02123159', 'tiger_cat', 0.0036861449)]}]
INFO:model_driver:Predictions took 2477.19 ms


([{'220px-Lynx_lynx_poing.jpg': [('n02127052', 'lynx', 0.98164833), ('n02128385', 'leopard', 0.0077441484), ('n02123159', 'tiger_cat', 0.0036861449)]}], 'Predictions took 2477.19 ms')
